## Time Series Interactive Tool

In this tutorial we will explore the different functionalities of Time Series Interactive Tool.

In [1]:
%matplotlib widget
from tvb.simulator.plot.timeseries_interactive import TimeSeriesInteractive

In [2]:
#Load the demo data
import numpy as np
data = np.load("demo_data_region_16s_2048Hz.npy")
period = 0.00048828125 #NOTE: Providing period in seconds

#Create a TVB TimeSeries object
import tvb.datatypes.time_series
tsr = tvb.datatypes.time_series.TimeSeriesRegion()
tsr.data = data
tsr.sample_period = period

from tvb.datatypes.connectivity import Connectivity
tsr.connectivity = Connectivity.from_file()


tsi = TimeSeriesInteractive(time_series=tsr)
tsi.configure()

2021-08-09 10:39:55,205 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.


In [3]:
tsi.show()

GridBox(children=(Output(layout=Layout(border='solid 1px black', margin='3px 3px 3px 3px', padding='2px 2px 2p…

As you can see in the output above, the control panel consists of Window Length Slider, Time Series Viewer Control Buttons, Spacing slider and Space Label Selection checkboxes.

### Let's explore each functionality one by one

##### Space Label Selection Checkboxes

<p>These checkboxes select/unselect the various space labels of the time series. Primary purpose of these checkboxes is to observe specific space labels clearly.</p>

<p><b>Try it out yourself:</b> Unselect all labels and select rA1, rA2, rIA and rIP only.</p>

##### Window Length Slider

<p>This slider determines the length of the window to observe the TimeSeries. The main use of this slider is to observe the small changes occuring in the Time Series in contrast to the overall waveform.</p>

<p><b>Try it out yourself:</b> Change the window length to 2, to observe the timeseries clearly.</p>

##### Spacing Slider

<p>This slider alters the spacing between two time series.</p>

<p><b>Try it out yourself:</b> Increase the spacing between different time series to 0.8 using the Spacing Slider.</p>

##### Time Series Viewer Buttons

These buttons help to navigate within the TimeSeries by moving the window by either 0.25 sec or 0.5 sec.
- The '<' and '>' buttons step 0.25 sec backward and forward in time respectively.
- The '<<' and '>>' buttons step 0.5 sec backward and forward in time respectively.
- The '|<<' and '>>|' buttons navigate to the beginning and end of the TimeSeries respectively.

<p><b>Try it out yourself:</b> Navigate forward in time and observe the Time Series.</p>